# Howamidoing Demo

How Am I Doing? A tool to calculate your final letter grade of a course that takes into the effect of curved grading policies, assignment drops, clobber policies, etc.

The hierarchy of the objects in this library can be viewed as:

- `Course`: the course, e.g. **MATH 100**.
  - `CurvedSingleAssignment`: an assignment that is graded on curved basis, e.g. **Midterm 1**.
  - `UncurvedSingleAssignment`: an assignment that is graded on absolute scale, e.g. **Class Participation**.
  - `CurvedAssignmentGroup`: a group of assignment that is all graded on a curved basis and is aggregated when calculating the score towards the final grade. E.g., **Projects**.
    - `Assignment`: **Project 1**.
    - ...
  - `UncurvedAssignmentGroup`: a group of assignment that is all graded on the absolute scale and is aggregated when calculating the score towards the final grade. E.g., **Homeworks**.
    - `Assignment`: **Homework 1**.
    - ...
  - ...

# Comprehensive Example

## Course Policy

| Component | Weight |
| --------- | ------ |
| Participation | 5% |
| Homeworks | 15% |
| Projects | 20% |
| Midterm 1 | 15% |
| Midterm 2 | 15% |
| Final | 30% |

- Participation: Uncurved.
- Homeworks: 10 in total, all uncurved. Allow dropping 2 lowest score.
- Projects: 4 in total, all curved. Allow dropping 1 lowest score (based on zscore).
- Midterm 1: Curved.
- Midterm 2: Curved.
- Final: Curved.

**Clobber Policy**: Your grade on the final can be clobbered for either midterms, but not both. 

## Your Scores

- Participation: 100/100
- Homeworks: [20/20, 19/20, 28/30, 23/25, 12/15, 20/20, 20/20, 0/30, 30/30, 19/21]
- Projects:
  - Project 1: 81/100. Class mean = 76, class standard deviation = 12.4
  - Project 2: 68/100. Class mean = 70, class standard deviation = 23.2
  - Project 3: 94/100. Class mean = 90, class standard deviation = 11.2
  - Project 1: 85/100. Class mean = 91, class standard deviation = 14.8
- Midterm 1: 79/130. Class mean = 84.3, class standard deviation = 24.6
- Midterm 2: 118/187. Class mean = 106, class standard deviation = 28.3
- Final: 186/300. Class mean = 174, class standard deviation = 26.5

Here is the example of how to calculate this complicated grades in `howamidoing`.

In [1]:
from howamidoing import *

math100 = Course(name="MATH100")
participation = math100.add_uncurved_single(0.05, 100)
hws = math100.add_uncurved_group(0.15, num_drops=2)
projects = math100.add_curved_group(0.2, num_drops=1)
mt1 = math100.add_curved_single(0.15, 79, upper=130, mu=84.3, sigma=24.6)
mt2 = math100.add_curved_single(0.15, 118, upper=187, mu=106, sigma=28.3)
final = math100.add_curved_single(0.3, 186, upper=300, mu=174, sigma=26.5)

In [2]:
hws.add_assignment(20, upper=20) # hw1
hws.add_assignment(19, upper=20) # hw2
hws.add_assignment(28, upper=30) # hw3
hws.add_assignment(23, upper=25) # hw4
hws.add_assignment(12, upper=15) # hw5
hws.add_assignment(20, upper=20) # hw6
hws.add_assignment(20, upper=20) # hw7
hws.add_assignment( 0, upper=20) # hw8
hws.add_assignment(30, upper=30) # hw9
hws.add_assignment(19, upper=21) # hw10

hws.get_detail()

{'score': 0.9635119047619047, 'stats': {}, 'drop_applied': True}

In [3]:
projects.add_assignment(81, mu=76, sigma=12.4) # proj1
projects.add_assignment(68, mu=70, sigma=23.2) # proj2
projects.add_assignment(94, mu=90, sigma=11.2) # proj3
projects.add_assignment(85, mu=91, sigma=14.8) # proj4

projects.get_detail()

{'score': 0.81,
 'stats': {'zscore': 0.17532948452466496,
  'mu': 0.7866666666666666,
  'sigma': 0.13308276925921692},
 'drop_applied': True}

In [4]:
print(math100.get_detail(), math100.get_grade())

{'score': 0.7283330382852442, 'stats': {'zscore': 0.2401665863037439, 'mu': 0.7081560877847642, 'sigma': 0.08401231333222071}, 'curved': True, 'is_final': True} B+


In [8]:
math100.apply_clobber(final.get_id(), targets=[mt1.get_id(), mt2.get_id()], capacity=1)
print("Detail: \n", math100.get_detail())
print("Letter grade:", math100.get_grade(show_boundary=True))

Detail: 
 {'score': 0.747301833640832, 'stats': {'zscore': 0.7489774426239227, 'mu': 0.7081560877847642, 'sigma': 0.05226558722373122}, 'curved': True, 'is_final': True}
Letter grade boundaries:
{'A+': 0.7941253228320848, 'A': 0.7467723506669821, 'A-': 0.7282950871521101, 'B+': 0.7015883230011577, 'B': 0.6807479865172447, 'B-': 0.6641682593529623, 'C+': 0.6539862876691098, 'C': 0.6411750423038068, 'C-': 0.631023002756979, 'D+': 0.6221868467770413, 'D': 0.6166554519410183, 'D-': 0.6098553051370617, 'F': 0.0}
Letter grade:  A


In [7]:
math100.corr = 0
print(math100.get_detail(), math100.get_grade(show_boundary=True))

Letter grade boundaries:
{'A+': 0.7941253228320848, 'A': 0.7467723506669821, 'A-': 0.7282950871521101, 'B+': 0.7015883230011577, 'B': 0.6807479865172447, 'B-': 0.6641682593529623, 'C+': 0.6539862876691098, 'C': 0.6411750423038068, 'C-': 0.631023002756979, 'D+': 0.6221868467770413, 'D': 0.6166554519410183, 'D-': 0.6098553051370617, 'F': 0.0}
{'score': 0.747301833640832, 'stats': {'zscore': 0.7489774426239227, 'mu': 0.7081560877847642, 'sigma': 0.05226558722373122}, 'curved': True, 'is_final': True} A
